In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from datetime import date
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [3]:
# Import data
file_path = Path("Resources/sp500_adj_close_raw.csv")
df = pd.read_csv(file_path)

In [4]:
# Import data
file_path = Path("Resources/sp500_adj_close_raw.csv")
df = pd.read_csv(file_path)
# Convert date to datetime data type
df["Date"] = pd.to_datetime(df["Date"])
df.set_index('Date', inplace=True)
df.shape
df.tail()

,Ticker,Adjusted Close,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
Date,,,,,,,,,,,,,,,
2024-10-25,XYL,130.41,130.41,130.65,-0.001837,0.009297,39.056873,133.51048,134.41891,130.84920,138.399480,129.94153,126.71,137.53000,short
2024-10-25,YUM,133.04,133.04,133.16,-0.000901,0.006858,34.395523,134.61107,133.44423,134.00830,139.507830,130.66417,129.71,139.92000,short
2024-10-25,ZBH,102.33,102.33,104.00,-0.016058,0.010610,50.197327,107.36608,107.96970,115.37445,107.723175,101.30882,101.77,115.91237,short
2024-10-25,ZBRA,359.97,359.97,362.05,-0.005745,0.009788,43.769238,355.89080,336.89050,311.04135,380.011440,359.82355,320.77,377.68000,short
2024-10-25,ZTS,179.91,179.91,181.50,-0.008760,0.012576,36.496883,189.09400,183.31490,179.24370,197.887830,182.27017,179.91,196.48000,short


In [5]:
# Feature engineering
df['Year'] = df.index.year
df['Month'] = df.index.month
df['Day'] = df.index.day
df['Day_of_Week'] = df.index.dayofweek


In [6]:
# Initialize the LabelEncoder and fit it to the Action column
le_action = LabelEncoder()
df['Action'] = le_action.fit_transform(df['Action'])

# Display the dataframe
df.tail()

,Ticker,Adjusted Close,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action,Year,Month,Day,Day_of_Week
Date,,,,,,,,,,,,,,,,,,,
2024-10-25,XYL,130.41,130.41,130.65,-0.001837,0.009297,39.056873,133.51048,134.41891,130.84920,138.399480,129.94153,126.71,137.53000,3,2024,10,25,4
2024-10-25,YUM,133.04,133.04,133.16,-0.000901,0.006858,34.395523,134.61107,133.44423,134.00830,139.507830,130.66417,129.71,139.92000,3,2024,10,25,4
2024-10-25,ZBH,102.33,102.33,104.00,-0.016058,0.010610,50.197327,107.36608,107.96970,115.37445,107.723175,101.30882,101.77,115.91237,3,2024,10,25,4
2024-10-25,ZBRA,359.97,359.97,362.05,-0.005745,0.009788,43.769238,355.89080,336.89050,311.04135,380.011440,359.82355,320.77,377.68000,3,2024,10,25,4
2024-10-25,ZTS,179.91,179.91,181.50,-0.008760,0.012576,36.496883,189.09400,183.31490,179.24370,197.887830,182.27017,179.91,196.48000,3,2024,10,25,4


In [7]:
# Define X and Y variables for train and test datasets
X = df.drop(columns=["Action", "Ticker", "Previous Day Close", "Resistance", "Upper Band", "SMA_50", "SMA_200", "Next Day Close"])
y = df["Action"]

# Preserve the Ticker column separately
tickers = df['Ticker']

In [8]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
# Create a StandardScaler model and fit it to the training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
# Define the parameter grid for GridSearchCV
param_grid = {'n_neighbors': np.arange(1, 30, 2)}


In [11]:
# Initialize and run GridSearchCV
grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
grid_knn.fit(X_train_scaled, y_train)


Fitting 5 folds for each of 15 candidates, totalling 75 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29])},
             scoring='accuracy', verbose=2)

In [12]:
# Best hyperparameters and corresponding score
print("Best Parameters:", grid_knn.best_params_)
print("Best Cross-Validation Score:", grid_knn.best_score_)


Best Parameters: {'n_neighbors': 17}
Best Cross-Validation Score: 0.6164262092295264


In [13]:
# Fit the model with the best found hyperparameters
best_knn_classifier = grid_knn.best_estimator_
best_knn_classifier.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=17)

In [14]:
# Predict on the test set using the best model
y_pred_class = best_knn_classifier.predict(X_test_scaled)

In [15]:
# Evaluate the classifier
print("Classification Report for Best KNN Classifier:")
print(classification_report(y_test, y_pred_class))

Classification Report for Best KNN Classifier:
              precision    recall  f1-score   support

           0       0.63      0.69      0.66     99062
           1       0.62      0.66      0.64    107964
           2       0.61      0.57      0.59     99478
           3       0.63      0.56      0.60     89616

    accuracy                           0.62    396120
   macro avg       0.62      0.62      0.62    396120
weighted avg       0.62      0.62      0.62    396120



In [16]:
# Generate and display the confusion matrix
conf_matrix_class = confusion_matrix(y_test, y_pred_class)
conf_df_class = pd.DataFrame(
    conf_matrix_class, 
    index=le_action.classes_, 
    columns=le_action.classes_
)
print("Confusion Matrix:")
print(conf_df_class)

Confusion Matrix:
         buy   hold   sell  short
buy    67870   4010   1592  25590
hold    3961  71693  31076   1234
sell    1988  38232  57112   2146
short  33713   2484   3132  50287


In [17]:
# Predict actions for the last day and export to CSV:
last_day_class = df.loc[df.index == df.index.max()].drop(columns=["Action", "Previous Day Close", "Resistance", "Upper Band", "SMA_50", "SMA_200", "Next Day Close"])


In [18]:
# Preserve the Ticker column from the last day
last_day_tickers = last_day_class['Ticker']

In [19]:
# Drop the Ticker column before scaling
X_last_day_class = last_day_class.drop(columns=["Ticker"])

In [20]:
# Standardize the last day data
last_day_scaled = scaler.transform(X_last_day_class)
predicted_actions = best_knn_classifier.predict(last_day_scaled)


In [21]:
# Add predictions and ticker back to the DataFrame
last_day_class['Predicted_Action'] = le_action.inverse_transform(predicted_actions)
last_day_class['Ticker'] = last_day_tickers

In [22]:
# Display the last day's predictions with Ticker
print(last_day_class[['Ticker', 'Adjusted Close', 'Predicted_Action']])

           Ticker  Adjusted Close Predicted_Action
Date                                              
2024-10-25      A          130.16              buy
2024-10-25   AAPL          231.40             sell
2024-10-25   ABBV          187.82              buy
2024-10-25   ABNB          134.60             hold
2024-10-25    ABT          114.24            short
...           ...             ...              ...
2024-10-25    XYL          130.41             sell
2024-10-25    YUM          133.04             hold
2024-10-25    ZBH          102.33            short
2024-10-25   ZBRA          359.97            short
2024-10-25    ZTS          179.91            short

[501 rows x 3 columns]


In [23]:
# Export the predictions
last_day_class.to_csv('predicted_actions_last_day_KNN_class.csv')